# Multimodalna Analiza Zmienności Rytmu Serca

**Autorzy:** Bartłomiej Tempka, Juliusz Wasieleski 
**Data:** Czerwiec 2025

## Opis projektu
Projekt polega na wykonaniu multimodalnej analizy zmienności rytmu serca (HRV) z wykorzystaniem trzech datasetów:
1. **Mechanocardiograms** - EKG + akcelerometria (Dataset 1)
2. **Cardiac patients with valvular diseases** - EKG + akcelerometria + żyroskopia (Dataset 2)  
3. **CEBSDB** - EKG + oddychanie + sejsmokardiografia (Dataset 3)

## Cele:
- Implementacja detektorów tętna dla różnych modalności
- Analiza HRV w dziedzinie czasu, częstotliwości i nieliniowa
- Porównanie wyników między modalnościami

In [26]:
# Importy podstawowe
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks, butter, filtfilt
import os
import warnings
warnings.filterwarnings('ignore')

# Konfiguracja wykresów
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10



## 1. Eksploracja Dataset 1: Mechanocardiograms

Ten dataset zawiera sygnały EKG wraz z akcelerometrią (mechanokardiografia). Sprawdźmy strukturę danych.


In [27]:
def load_mechanocardiogram(filepath):
    """Wczytanie danych mechanokardiogramów z pliku txt"""
    data = {}
    signals = []
    
    with open(filepath, 'r') as file:
        lines = file.readlines()
    
    header_end = 0
    for i, line in enumerate(lines):
        if line.startswith('[DATA]'):
            header_end = i + 1
            break
        elif 'Sample rate' in line:
            data['fs'] = int(line.split('=')[1].strip().replace('Hz', ''))
        elif 'Measurement length' in line:
            data['duration'] = line.split(':')[1].strip()
        elif line.startswith('Signal'):
            signal_info = line.strip().split(': ')
            signal_name = signal_info[1].split(',')[0]
            signals.append(signal_name)
    
    # Wczytaj dane numeryczne
    if header_end > 0:
        numeric_data = []
        for line in lines[header_end:]:
            if line.strip():
                values = [float(x) for x in line.strip().split()]
                numeric_data.append(values)
        
        data['signals'] = np.array(numeric_data).T
        data['signal_names'] = signals
        data['time'] = np.arange(len(data['signals'][0])) / data['fs']

    for signal_name in signals:
        data[signal_name] = data['signals'][signals.index(signal_name)]
    
    data.pop('signals', None)  # Usunięcie surowych danych sygnałów z głównego słownika
    data.pop('signal_names', None)  # Usunięcie listy nazw sygnałów
    if 'EKG' in data:
        data['ecg'] = data.pop('EKG', None) 
    elif 'ECG' in data:
        data['ecg'] = data.pop('ECG', None)
    else:
        raise ValueError("Brak danych w pliku lub nieprawidłowy format pliku.")
    
    return data

print(load_mechanocardiogram('Surowe/sub_1.txt'))



{'fs': 800, 'duration': '3 min', 'time': array([0.0000000e+00, 1.2500000e-03, 2.5000000e-03, ..., 1.8335625e+02,
       1.8335750e+02, 1.8335875e+02], shape=(146688,)), 'accX': array([6080., -712., -688., ..., -312., -344., -368.], shape=(146688,)), 'accY': array([16892., -1160., -1056., ..., -1400., -1416., -1432.],
      shape=(146688,)), 'accZ': array([ -1704., -16448., -16432., ..., -16312., -16320., -16352.],
      shape=(146688,)), 'gyroX': array([    0.,  1024.,  1024., ..., 19712., 19456., 19712.],
      shape=(146688,)), 'gyroY': array([    0.,  1536.,  1792., ..., -7681., -7681., -5377.],
      shape=(146688,)), 'gyroZ': array([   0., -256.,    0., ..., 8704., 9728., 9472.], shape=(146688,)), 'ecg': array([-436.,  446., -553., ..., -824.,  807.,  542.], shape=(146688,))}


## 2. Eksploracja Dataset 2: Cardiac Patients

Ten dataset zawiera dane pacjentów z wadami zastawkowymi serca, rejestrowane za pomocą sensorów Shimmer.


In [ ]:
def load_cardiac_patient_data(filepath):
    """Wczytanie danych pacjentów kardiologicznych z pliku CSV Shimmer"""
    
    # Wczytaj plik CSV, pomijając pierwszą linię z sep=,
    #get separator from the first line
    with open(filepath, 'r') as f:
        first_line = f.readline().strip()
        #remove ""
        first_line = first_line.replace('"', '')
        if first_line.startswith('sep='):
            skiprows = 1
        else:
            skiprows = 0
        sep = first_line.split('=')[1] if '=' in first_line else ','
    # Wczytaj dane z pliku CSV
    df = pd.read_csv(filepath, skiprows=skiprows, sep=sep)
    # Wyodrębnij podstawowe informacje
    data = {}
    
    # POPRAWKA: Konwertuj timestamp na liczbę
    timestamp_raw = df.iloc[:, 0].values  # Pierwsza kolumna to timestamp
    data['timestamp'] = pd.to_numeric(timestamp_raw, errors='coerce')
    
    # Usuń wiersze z nieprawidłowymi timestampami (NaN)
    valid_mask = ~np.isnan(data['timestamp'])
    data['timestamp'] = data['timestamp'][valid_mask]
    
    # Identyfikuj kolumny według nazw
    ecg_cols = [col for col in df.columns if 'ECG' in col and 'CAL' in col]
    acc_cols = [col for col in df.columns if 'Accel' in col and 'CAL' in col]
    gyro_cols = [col for col in df.columns if 'Gyro' in col and 'CAL' in col]
    
    if ecg_cols:
        ecg_raw = df[ecg_cols].values[valid_mask]
        data['ecg'] = pd.DataFrame(ecg_raw).apply(pd.to_numeric, errors='coerce').values
        data['ecg_names'] = ecg_cols
    
    if acc_cols:
        acc_raw = df[acc_cols].values[valid_mask]
        data['accelerometer'] = pd.DataFrame(acc_raw).apply(pd.to_numeric, errors='coerce').values
        data['acc_names'] = acc_cols
        
    if gyro_cols:
        gyro_raw = df[gyro_cols].values[valid_mask]
        data['gyroscope'] = pd.DataFrame(gyro_raw).apply(pd.to_numeric, errors='coerce').values
        data['gyro_names'] = gyro_cols
    
    # Usuń wiersze z NaN w sygnałach (jeśli występują)
    if 'ecg' in data:
        data['ecg'] = data['ecg'].astype(float)
        ecg_valid = ~np.isnan(data['ecg']).any(axis=1)
        for key in ['timestamp', 'ecg', 'accelerometer', 'gyroscope']:
            if key in data and key != 'timestamp':
                data[key] = data[key][ecg_valid]
        if 'timestamp' in data:
            data['timestamp'] = data['timestamp'][ecg_valid]
    
    # Oszacuj częstotliwość próbkowania
    if len(data['timestamp']) > 1:
        dt = np.median(np.diff(data['timestamp']))
        data['fs'] = 1.0 / (dt / 1000.0)  # Convert ms to Hz
        if data['fs'] == np.inf:
            data['fs'] = 256.0
    
    # Czas w sekundach od początku
    data['time'] = (data['timestamp'] - data['timestamp'][0]) / 1000.0
    
    # Sprawdź typy danych
    # if 'ecg' in data:
    #     print(f"Typ danych EKG: {data['ecg'].dtype}")
    # if 'accelerometer' in data:
    #     print(f"Typ danych akcelerometru: {data['accelerometer'].dtype}")
    # if 'gyroscope' in data:
    #     print(f"Typ danych żyroskopu: {data['gyroscope'].dtype}")
    

    data['accX'] = data['accelerometer'][:, 0] if 'accelerometer' in data else None
    data['accY'] = data['accelerometer'][:, 1] if 'accelerometer' in data else None
    data['accZ'] = data['accelerometer'][:, 2] if 'accelerometer' in data else None
    data['gyroX'] = data['gyroscope'][:, 0] if 'gyroscope' in data else None
    data['gyroY'] = data['gyroscope'][:, 1] if 'gyroscope' in data else None
    data['gyroZ'] = data['gyroscope'][:, 2] if 'gyroscope' in data else None
    for i in range(len(data['ecg_names'])):
        if "LA-RA" in data['ecg_names'][i]:
            data['ecg'] = data['ecg'][:, i]
    
    # ECG_ECG_LA-RA_24BIT_CAL
    # data['ecg'] = data['ecg'][:, 0] if 'ecg' in data else None  # Użyj tylko pierwszej kolumny EKG TODO: check bo to chyba status???

    data.pop('accelerometer', None)  # Usunięcie surowych danych akcelerometru
    data.pop('gyroscope', None)  # Usunięcie surowych danych żyroskopu
    data.pop('acc_names', None)  # Usunięcie listy nazw akcelerometru
    data.pop('gyro_names', None)  # Usunięcie listy nazw żyroskopu
    data.pop('ecg_names', None)  # Usunięcie listy nazw EKG
    return data
        
 
# data = load_cardiac_patient_data('Raw_Recordings/UP-25-Raw.csv')
# for key in data.keys():
#     print(f"{key}: {data[key].shape if isinstance(data[key], np.ndarray) else data[key]}")



## 3. Implementacja Detektorów Tętna

Implementujemy detektory dla różnych modalności:
- **EKG**: Klasyczny detektor zespołów QRS
- **SCG** (Sejsmokardiografia): Detektor na podstawie akcelerometru
- **GCG** (Żyrokardiografia): Detektor na podstawie żyroskopu


In [29]:
class HeartbeatDetector:
    """Klasa bazowa dla detektorów tętna"""
    
    def __init__(self, fs):
        self.fs = fs
    
    def bandpass_filter(self, signal, low_freq, high_freq, order=4):
        """Filtr pasmowo-przepustowy"""
        nyquist = self.fs / 2
        low = low_freq / nyquist
        high = high_freq / nyquist
        b, a = butter(order, [low, high], btype='band')
        return filtfilt(b, a, signal)
    
    def detect_peaks(self, signal, **kwargs):
        """Bazowa metoda detekcji pików"""
        raise NotImplementedError

class ECGDetector(HeartbeatDetector):
    """Detektor dla sygnałów EKG - Pan-Tompkins algorithm"""
    
    def detect_peaks(self, ecg_signal, min_peak_height=None, min_distance=None):
        """Detekcja zespołów QRS w sygnale EKG"""
        
        # Parametry domyślne
        if min_distance is None:
            min_distance = int(0.6 * self.fs)  # Minimum 100 BPM
        
        # Krok 1: Filtracja pasmowa (5-15 Hz dla QRS)
        filtered = self.bandpass_filter(ecg_signal, 5, 15)
        
        # Krok 2: Różniczkowanie
        diff_signal = np.diff(filtered)
        
        # Krok 3: Podnoszenie do kwadratu
        squared = diff_signal ** 2
        
        # Krok 4: Średnia ruchoma
        window_size = int(0.15 * self.fs)  # 150ms window
        integrated = np.convolve(squared, np.ones(window_size)/window_size, mode='same')
        
        # Krok 5: Detekcja pików
        if min_peak_height is None:
            min_peak_height = 0.35 * np.max(integrated)
        
        peaks, _ = find_peaks(integrated, 
                            height=min_peak_height,
                            distance=min_distance)
        
        return peaks

class SCGDetector(HeartbeatDetector):
    """Detektor dla sejsmokardiogramów (akcelerometr)"""
    
    def detect_peaks(self, acc_signal, axis=2, min_peak_height=None, min_distance=None):
        """Detekcja uderzeń serca z sygnału akcelerometru"""
        
        if min_distance is None:
            min_distance = int(0.4 * self.fs)  # Minimum 150 BPM
        
        # Wybierz odpowiednią oś lub oblicz magnitude
        if acc_signal.ndim == 1:
            signal_to_process = acc_signal
        else:
            if axis is None:
                # Magnitude wszystkich osi
                signal_to_process = np.sqrt(np.sum(acc_signal**2, axis=1))
            else:
                signal_to_process = acc_signal[:, axis]
        
        # Filtracja dla sygnałów kardiologicznych (1-40 Hz)
        filtered = self.bandpass_filter(signal_to_process, 1, 40)
        
        # Wzmocnienie przez podniesienie do kwadratu
        enhanced = filtered ** 2
        
        # Wygładzenie
        window_size = int(0.1 * self.fs)  # 100ms window
        smoothed = np.convolve(enhanced, np.ones(window_size)/window_size, mode='same')
        
        # Detekcja pików
        if min_peak_height is None:
            min_peak_height = 0.3 * np.max(smoothed)
        
        peaks, _ = find_peaks(smoothed,
                            height=min_peak_height,
                            distance=min_distance)
        
        return peaks

class GCGDetector(HeartbeatDetector):
    """Detektor dla żyrokardiogramów (żyroskop)"""
    
    def detect_peaks(self, gyro_signal, axis=2, min_peak_height=None, min_distance=None):
        """Detekcja uderzeń serca z sygnału żyroskopu"""
        
        if min_distance is None:
            min_distance = int(0.4 * self.fs)  # Minimum 150 BPM
        
        # Wybierz odpowiednią oś lub oblicz magnitude
        if gyro_signal.ndim == 1:
            signal_to_process = gyro_signal
        else:
            if axis is None:
                # Magnitude wszystkich osi
                signal_to_process = np.sqrt(np.sum(gyro_signal**2, axis=1))
            else:
                signal_to_process = gyro_signal[:, axis]
        
        # Filtracja dla sygnałów kardiologicznych (1-20 Hz)
        filtered = self.bandpass_filter(signal_to_process, 1, 20)
        
        # Wartość bezwzględna różniczki (wykrywa szybkie zmiany)
        diff_signal = np.abs(np.diff(filtered))
        diff_signal = np.append(diff_signal, diff_signal[-1])  # Restore length
        
        # Wygładzenie
        window_size = int(0.08 * self.fs)  # 80ms window
        smoothed = np.convolve(diff_signal, np.ones(window_size)/window_size, mode='same')
        
        # Detekcja pików
        if min_peak_height is None:
            min_peak_height = 0.4 * np.max(smoothed)
        
        peaks, _ = find_peaks(smoothed,
                            height=min_peak_height,
                            distance=min_distance)
        
        return peaks


## 4. Analiza HRV (Heart Rate Variability)

Implementujemy analizę zmienności rytmu serca w trzech domenach:
- **Dziedzina czasu**: AVNN, SDNN, RMSSD, pNN50
- **Dziedzina częstotliwości**: VLF, LF, HF, LF/HF
- **Analiza nieliniowa**: wykres Poincaré (SD1, SD2)


In [30]:
class HRVAnalyzer:
    """Klasa do analizy zmienności rytmu serca"""
    
    def __init__(self, rr_intervals):
        """
        Args:
            rr_intervals: odstępy RR w milisekundach
        """
        self.rr_intervals = np.array(rr_intervals)
        self.nn_intervals = self.filter_normal_intervals()  # Filtrowane odstępy NN
    
    def filter_normal_intervals(self, min_rr=300, max_rr=2000):
        """Filtracja nietypowych odstępów RR"""
        mask = (self.rr_intervals >= min_rr) & (self.rr_intervals <= max_rr)
        return self.rr_intervals[mask]
    
    def time_domain_analysis(self):
        """Analiza w dziedzinie czasu"""
        if len(self.nn_intervals) < 2:
            return {}
        
        # Podstawowe statystyki
        avnn = np.mean(self.nn_intervals)  # Średni odstęp NN
        sdnn = np.std(self.nn_intervals, ddof=1)  # Odchylenie standardowe
        
        # RMSSD - Root Mean Square of Successive Differences
        diff_nn = np.diff(self.nn_intervals)
        rmssd = np.sqrt(np.mean(diff_nn**2))
        
        # pNN50 - procent różnic > 50ms
        pnn50 = (np.sum(np.abs(diff_nn) > 50) / len(diff_nn)) * 100
        
        # pNN20 - procent różnic > 20ms (dodatkowy wskaźnik)
        pnn20 = (np.sum(np.abs(diff_nn) > 20) / len(diff_nn)) * 100
        
        return {
            'AVNN': avnn,
            'SDNN': sdnn,
            'RMSSD': rmssd,
            'pNN50': pnn50,
            'pNN20': pnn20,
            'HR_mean': 60000 / avnn if avnn > 0 else 0  # Średnie tętno
        }
    
    def frequency_domain_analysis(self, fs_resample=4):
        """Analiza w dziedzinie częstotliwości"""
        if len(self.nn_intervals) < 10:
            return {}
        
        # Interpolacja do równomiernie próbkowanego sygnału
        time_original = np.cumsum(self.nn_intervals) / 1000.0  # Convert to seconds
        time_new = np.arange(0, time_original[-1], 1/fs_resample)
        
        # Interpolacja liniowa
        rr_interpolated = np.interp(time_new, time_original, self.nn_intervals)
        
        # Detrending
        rr_detrended = rr_interpolated - np.mean(rr_interpolated)
        
        # FFT
        fft_vals = np.fft.fft(rr_detrended)
        fft_freqs = np.fft.fftfreq(len(rr_detrended), 1/fs_resample)
        
        # Power spectral density (PSD)
        psd = np.abs(fft_vals)**2
        
        # Definicje pasm częstotliwości (Hz)
        vlf_band = (0.0033, 0.04)
        lf_band = (0.04, 0.15)
        hf_band = (0.15, 0.4)
        
        # Znajdź indeksy dla każdego pasma
        def get_band_power(freq_range):
            mask = (fft_freqs >= freq_range[0]) & (fft_freqs <= freq_range[1])
            return np.sum(psd[mask]) if np.any(mask) else 0
        
        vlf_power = get_band_power(vlf_band)
        lf_power = get_band_power(lf_band)
        hf_power = get_band_power(hf_band)
        
        total_power = vlf_power + lf_power + hf_power
        
        # Znormalizowane moce
        lf_norm = (lf_power / (lf_power + hf_power)) * 100 if (lf_power + hf_power) > 0 else 0
        hf_norm = (hf_power / (lf_power + hf_power)) * 100 if (lf_power + hf_power) > 0 else 0
        
        # Stosunek LF/HF
        lf_hf_ratio = lf_power / hf_power if hf_power > 0 else 0
        
        return {
            'VLF_power': vlf_power,
            'LF_power': lf_power,
            'HF_power': hf_power,
            'Total_power': total_power,
            'LF_norm': lf_norm,
            'HF_norm': hf_norm,
            'LF_HF_ratio': lf_hf_ratio
        }
    
    def nonlinear_analysis(self):
        """Analiza nieliniowa - wykres Poincaré"""
        if len(self.nn_intervals) < 2:
            return {}
        
        # Współrzędne wykresu Poincaré
        rr_n = self.nn_intervals[:-1]
        rr_n1 = self.nn_intervals[1:]
        
        # SD1 - krótkookresowa zmienność (szerokość chmury punktów)
        diff_rr = rr_n1 - rr_n
        sd1 = np.std(diff_rr, ddof=1) / np.sqrt(2)
        
        # SD2 - długookresowa zmienność (długość chmury punktów)
        sum_rr = rr_n1 + rr_n
        sd2 = np.std(sum_rr, ddof=1) / np.sqrt(2)
        
        # SD1/SD2 ratio
        sd_ratio = sd1 / sd2 if sd2 > 0 else 0
        
        return {
            'SD1': sd1,
            'SD2': sd2,
            'SD1_SD2_ratio': sd_ratio,
            'poincare_rr_n': rr_n,
            'poincare_rr_n1': rr_n1
        }
    
    def comprehensive_analysis(self):
        """Kompletna analiza HRV"""
        results = {}
        results.update(self.time_domain_analysis())
        results.update(self.frequency_domain_analysis())
        results.update(self.nonlinear_analysis())
        return results

def peaks_to_rr_intervals(peaks, fs):
    """Konwersja pozycji pików na odstępy RR w milisekundach"""
    if len(peaks) < 2:
        return np.array([])
    
    rr_samples = np.diff(peaks)
    rr_ms = (rr_samples / fs) * 1000  # Convert to milliseconds
    return rr_ms



## 5. Przykład Analizy Multimodalnej

Teraz przeprowadzimy przykładową analizę na rzeczywistych danych z obu datasetów.


In [31]:
def multimodal_analysis_example():
    """Przykład analizy multimodalnej dla jednego pacjenta"""
    
    try:
        # Załaduj dane pacjenta
        patient_data = load_cardiac_patient_data('Raw_Recordings/CP-01-Raw.csv')
        
        if patient_data is None:
            print("Nie udało się załadować danych pacjenta")
            return
        
        fs = patient_data['fs']
        duration_minutes = patient_data['time'][-1] / 60
        
        print(f"=== Analiza Pacjenta CP-01 ===")
        print(f"Czas nagrania: {duration_minutes:.1f} minut")
        print(f"Częstotliwość próbkowania: {fs:.1f} Hz")
        print()
        
        # Wybierz fragment danych (np. pierwsze 5 minut)
        max_samples = int(5 * 60 * fs)  # 5 minut
        end_idx = min(max_samples, len(patient_data['time']))
        
        # === ANALIZA EKG ===
        if 'ecg' in patient_data:
            print("--- Analiza EKG ---")
            # Użyj pierwszego odprowadzenia EKG
            ecg_signal = patient_data['ecg'][:end_idx]
            
            # Detekcja QRS
            ecg_detector = ECGDetector(fs)
            qrs_peaks = ecg_detector.detect_peaks(ecg_signal)
            
            if len(qrs_peaks) > 1:
                # Analiza HRV
                rr_intervals = peaks_to_rr_intervals(qrs_peaks, fs)
                hrv_analyzer = HRVAnalyzer(rr_intervals)
                hrv_results_ecg = hrv_analyzer.comprehensive_analysis()
                
                print(f"Wykryto {len(qrs_peaks)} zespołów QRS")
                print(f"Średnie tętno: {hrv_results_ecg.get('HR_mean', 0):.1f} BPM")
                print(f"SDNN: {hrv_results_ecg.get('SDNN', 0):.1f} ms")
                print(f"RMSSD: {hrv_results_ecg.get('RMSSD', 0):.1f} ms")
                print(f"LF/HF: {hrv_results_ecg.get('LF_HF_ratio', 0):.2f}")
            else:
                print("Nie wykryto wystarczającej liczby zespołów QRS")
                hrv_results_ecg = {}

        # === ANALIZA AKCELEROMETRU ===
        if 'accZ' in patient_data:
            print("\n--- Analiza Akcelerometru ---")
            # Użyj osi Z akcelerometru
            acc_signal = patient_data['accZ'][:end_idx] # Z-axis
            
            # Detekcja uderzeń serca
            scg_detector = SCGDetector(fs)
            scg_peaks = scg_detector.detect_peaks(acc_signal, axis=None)  # Single axis
            
            if len(scg_peaks) > 1:
                # Analiza HRV
                rr_intervals_scg = peaks_to_rr_intervals(scg_peaks, fs)
                hrv_analyzer_scg = HRVAnalyzer(rr_intervals_scg)
                hrv_results_scg = hrv_analyzer_scg.comprehensive_analysis()
                
                print(f"Wykryto {len(scg_peaks)} uderzeń serca (SCG)")
                print(f"Średnie tętno: {hrv_results_scg.get('HR_mean', 0):.1f} BPM")
                print(f"SDNN: {hrv_results_scg.get('SDNN', 0):.1f} ms")
                print(f"RMSSD: {hrv_results_scg.get('RMSSD', 0):.1f} ms")
                print(f"LF/HF: {hrv_results_scg.get('LF_HF_ratio', 0):.2f}")
            else:
                print("Nie wykryto wystarczającej liczby uderzeń serca (SCG)")
                hrv_results_scg = {}
        
        # === ANALIZA ŻYROSKOPU ===
        if 'gyroZ' in patient_data:
            print("\n--- Analiza Żyroskopu ---")
            # Użyj osi Z żyroskopu
            gyro_signal = patient_data['gyroZ'][:end_idx]  # Z-axis
            
            # Detekcja uderzeń serca
            gcg_detector = GCGDetector(fs)
            gcg_peaks = gcg_detector.detect_peaks(gyro_signal, axis=None)  # Single axis
            
            if len(gcg_peaks) > 1:
                # Analiza HRV
                rr_intervals_gcg = peaks_to_rr_intervals(gcg_peaks, fs)
                hrv_analyzer_gcg = HRVAnalyzer(rr_intervals_gcg)
                hrv_results_gcg = hrv_analyzer_gcg.comprehensive_analysis()
                
                print(f"Wykryto {len(gcg_peaks)} uderzeń serca (GCG)")
                print(f"Średnie tętno: {hrv_results_gcg.get('HR_mean', 0):.1f} BPM")
                print(f"SDNN: {hrv_results_gcg.get('SDNN', 0):.1f} ms")
                print(f"RMSSD: {hrv_results_gcg.get('RMSSD', 0):.1f} ms")
                print(f"LF/HF: {hrv_results_gcg.get('LF_HF_ratio', 0):.2f}")
            else:
                print("Nie wykryto wystarczającej liczby uderzeń serca (GCG)")
                hrv_results_gcg = {}
        
        print("\n=== Analiza zakończona ===")
        
    except Exception as e:
        print(f"Błąd podczas analizy: {e}")
        import traceback
        traceback.print_exc()


multimodal_analysis_example()


Błąd podczas analizy: too many indices for array: array is 1-dimensional, but 2 were indexed


Traceback (most recent call last):
  File "/tmp/ipykernel_63671/3618820127.py", line 6, in multimodal_analysis_example
    patient_data = load_cardiac_patient_data('Raw_Recordings/CP-01-Raw.csv')
  File "/tmp/ipykernel_63671/101613974.py", line 85, in load_cardiac_patient_data
    data['ecg'] = data['ecg'][:, i]
IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed


# 6. Pozyskanie Danych wszystkich deskrytporów

In [32]:

def get_descriptors(patient_data):
    result = {'ecg_pulse': 0.0,
              'ecg_sdnn': 0.0,
              'ecg_rmssd': 0.0,
              'ecg_lf_hf': 0.0,
              'accX_pulse': 0.0,
              'accX_sdnn': 0.0,
              'accX_rmssd': 0.0,
              'accX_lf_hf': 0.0,
              'accY_pulse': 0.0,
              'accY_sdnn': 0.0,
              'accY_rmssd': 0.0,
              'accY_lf_hf': 0.0,
              'accZ_pulse': 0.0,
              'accZ_sdnn': 0.0,
              'accZ_rmssd': 0.0,
              'accZ_lf_hf': 0.0,
              'gyroX_pulse': 0.0,
              'gyroX_sdnn': 0.0,
              'gyroX_rmssd': 0.0,
              'gyroX_lf_hf': 0.0,
              'gyroY_pulse': 0.0,
              'gyroY_sdnn': 0.0,
              'gyroY_rmssd': 0.0,
              'gyroY_lf_hf': 0.0,
              'gyroZ_pulse': 0.0,
              'gyroZ_sdnn': 0.0,
              'gyroZ_rmssd': 0.0,
              'gyroZ_lf_hf': 0.0}

    if patient_data is None or 'fs' not in patient_data or 'time' not in patient_data:
        print("Nieprawidłowe dane pacjenta lub brak wymaganych kluczy.")
        return result
    fs = patient_data['fs']
    duration_minutes = patient_data['time'][-1] / 60
    max_samples = int(duration_minutes * 60 * fs)  # 2 minut wiekszosc danych jest tej dlugosci
    end_idx = len(patient_data['time'])

    if 'ecg' in patient_data:
        ecg_signal = patient_data['ecg'][:end_idx]
        
        # Detekcja QRS
        ecg_detector = ECGDetector(fs)
        qrs_peaks = ecg_detector.detect_peaks(ecg_signal)
        
        if len(qrs_peaks) > 1:
            # Analiza HRV
            rr_intervals = peaks_to_rr_intervals(qrs_peaks, fs)
            hrv_analyzer = HRVAnalyzer(rr_intervals)
            hrv_results_ecg = hrv_analyzer.comprehensive_analysis()

            result['ecg_pulse'] = hrv_results_ecg.get('HR_mean', 0)
            result['ecg_sdnn'] = hrv_results_ecg.get('SDNN', 0)
            result['ecg_rmssd'] = hrv_results_ecg.get('RMSSD', 0)
            result['ecg_lf_hf'] = hrv_results_ecg.get('LF_HF_ratio', 0)

    for acc in ['accX', 'accY', 'accZ']:
        if acc in patient_data:
            acc_signal = patient_data[acc][:end_idx] 
            
            # Detekcja uderzeń serca
            scg_detector = SCGDetector(fs)
            scg_peaks = scg_detector.detect_peaks(acc_signal, axis=None)  # Single axis
            
            if len(scg_peaks) > 1:
                # Analiza HRV
                rr_intervals_scg = peaks_to_rr_intervals(scg_peaks, fs)
                hrv_analyzer_scg = HRVAnalyzer(rr_intervals_scg)
                hrv_results_scg = hrv_analyzer_scg.comprehensive_analysis()

                result[f'{acc}_pulse'] = hrv_results_scg.get('HR_mean', 0)
                result[f'{acc}_sdnn'] = hrv_results_scg.get('SDNN', 0)
                result[f'{acc}_rmssd'] = hrv_results_scg.get('RMSSD', 0)
                result[f'{acc}_lf_hf'] = hrv_results_scg.get('LF_HF_ratio', 0)
    for gyro in ['gyroX', 'gyroY', 'gyroZ']:
        if gyro in patient_data:
            gyro_signal = patient_data[gyro][:end_idx]  
            
            # Detekcja uderzeń serca
            gcg_detector = GCGDetector(fs)
            gcg_peaks = gcg_detector.detect_peaks(gyro_signal, axis=None)
            if len(gcg_peaks) > 1:
                # Analiza HRV
                rr_intervals_gcg = peaks_to_rr_intervals(gcg_peaks, fs)
                hrv_analyzer_gcg = HRVAnalyzer(rr_intervals_gcg)
                hrv_results_gcg = hrv_analyzer_gcg.comprehensive_analysis()

                result[f'{gyro}_pulse'] = hrv_results_gcg.get('HR_mean', 0)
                result[f'{gyro}_sdnn'] = hrv_results_gcg.get('SDNN', 0)
                result[f'{gyro}_rmssd'] = hrv_results_gcg.get('RMSSD', 0)
                result[f'{gyro}_lf_hf'] = hrv_results_gcg.get('LF_HF_ratio', 0)
    return result




In [33]:
#load all data
import os
healthy_directpor = 'Surowe'
sick_directpor = 'Raw_Recordings'
df = pd.DataFrame()
for filename in os.listdir(healthy_directpor):
    print(f"Przetwarzanie pliku: {filename}")
    data = load_mechanocardiogram(os.path.join(healthy_directpor, filename))
    descriptors = get_descriptors(data)
    descriptors['sick'] = 0
    #save descriptors to csv
    df = pd.concat([df, pd.DataFrame([descriptors])], ignore_index=True)

for filename in os.listdir(sick_directpor):
    print(f"Przetwarzanie pliku: {filename}")
    data = load_cardiac_patient_data(os.path.join(sick_directpor, filename))
    if data is not None:
        descriptors = get_descriptors(data)
        descriptors['sick'] = 1
        #save descriptors to csv
        df = pd.concat([df, pd.DataFrame([descriptors])], ignore_index=True)
# Save the descriptors to a CSV file
df.to_csv('descriptors.csv', index=False)

Przetwarzanie pliku: sub_10.txt
Przetwarzanie pliku: sub_8.txt
Przetwarzanie pliku: sub_18.txt
Przetwarzanie pliku: sub_14.txt
Przetwarzanie pliku: sub_2.txt
Przetwarzanie pliku: sub_11.txt
Przetwarzanie pliku: sub_22.txt
Przetwarzanie pliku: sub_21.txt
Przetwarzanie pliku: sub_16.txt
Przetwarzanie pliku: sub_15.txt
Przetwarzanie pliku: sub_24.txt
Przetwarzanie pliku: sub_12.txt
Przetwarzanie pliku: sub_28.txt
Przetwarzanie pliku: sub_3.txt
Przetwarzanie pliku: sub_20.txt
Przetwarzanie pliku: sub_4.txt
Przetwarzanie pliku: sub_9.txt
Przetwarzanie pliku: sub_7.txt
Przetwarzanie pliku: sub_6.txt
Przetwarzanie pliku: sub_17.txt
Przetwarzanie pliku: sub_23.txt
Przetwarzanie pliku: sub_29.txt
Przetwarzanie pliku: sub_25.txt
Przetwarzanie pliku: sub_27.txt
Przetwarzanie pliku: sub_19.txt
Przetwarzanie pliku: sub_13.txt
Przetwarzanie pliku: sub_26.txt
Przetwarzanie pliku: sub_5.txt
Przetwarzanie pliku: sub_1.txt
Przetwarzanie pliku: CP-26-Raw.csv


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed